In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import requests
from talib import abstract
import math

plt.style.use('fivethirtyeight')

In [2]:
data_request = requests.get('https://api3.binance.com/api/v3/klines?symbol=BNBBUSD&interval=5m&limit=1000')
bnb_btc_data = data_request.json()
data = pd.DataFrame(data = {
    'open time': [],
    'open': [],
    'high': [],
    'low': [],
    'close': [],
})

for kline_item in bnb_btc_data :
    time = kline_item[0] / 1000
    row = {
        'open time': datetime.utcfromtimestamp(time),
        'open': float(kline_item[1]),
        'high': float(kline_item[2]),
        'low': float(kline_item[3]),
        'close': float(kline_item[4]),
    }
    data = data.append(row, ignore_index=True)


data.head()

,open time,open,high,low,close
0,2021-10-31 04:40:00,533.9,534.9,533.8,534.7
1,2021-10-31 04:45:00,534.7,535.0,532.7,532.9
2,2021-10-31 04:50:00,533.0,534.0,533.0,533.9
3,2021-10-31 04:55:00,533.9,534.8,533.8,534.4
4,2021-10-31 05:00:00,534.3,535.0,533.3,533.7


In [3]:
data = pd.read_csv('BNBBUSD-5m.csv')
# data = data.iloc[-40880:-1]
# data = data.reset_index()
# data

In [4]:
data['sma50'] = abstract.SMA(data, timeperiod=50)
data['sma20'] = abstract.SMA(data, timeperiod=20)
data['sma150'] = abstract.SMA(data, timeperiod=150)
macd = abstract.MACD(data, fastperiod=12, slowperiod=26, signalperiod=9)
data['macd'] = macd['macd']
data['macdsignal'] = macd['macdsignal']
data['macdhist'] = macd['macdhist']
data['roc'] = abstract.ROC(data, timeperiod=200)
data['rsi'] = abstract.RSI(data, timeperiod=28)
data['sar'] = abstract.SAR(data['high'], data['low'], acceleration=0.02, maximum=0.02)
a = data[['open time', 'close', 'sma150', 'sar', 'roc', 'macd', 'macdsignal', 'macdhist']].copy()
data.to_csv("data.csv", index=False)

In [17]:
money = 1000
sigPriceBuy = []
sigPriceSell = []

earns = pd.DataFrame(data = {
    'buy time': [],
    'buy': [],
    'sell': [],
    'sell time': [],
    'earn': [],
})

flag = 'sold'

for i in range(len(data)):
    if i <= 150:
        sigPriceBuy.append(np.nan)
        sigPriceSell.append(np.nan)
        continue
        
    
    initial_value = data.iloc[i-150]
    current_value = data.iloc[i]
        
        
        
        
    if flag == 'bought':
        preenchidos = earns[earns['buy'].notna() & earns['sell'].isna()]
        
        
#         last_row = preenchidos.iloc[-1]['buy time']
#         minutes_diff = (current_value['open time'] - last_row).total_seconds() / 60
        
        last_row = preenchidos.iloc[-1]['buy time']
        sell_time = datetime.strptime(last_row, '%Y-%m-%d %H:%M:%S')
        current_rime = datetime.strptime(current_value['open time'], '%Y-%m-%d %H:%M:%S')
        minutes_diff = (current_rime - sell_time).total_seconds() / 60
        
        if minutes_diff < 10:
            sigPriceBuy.append(np.nan)
            sigPriceSell.append(np.nan)
            continue

        
        rowIndex = earns[earns['buy'].notna() & earns['sell'].isna()].index.item()
        variation = (data['close'][i] / earns['buy'][rowIndex]) - 1
        
        if data['rsi'][i] > 70 or variation <= -0.02:
            sigPriceBuy.append(np.nan)
            sigPriceSell.append(data['close'][i])
            flag = 'sold'
            earns.loc[rowIndex, 'sell'] = data['close'][i]
            earns.loc[rowIndex, 'earn'] = data['close'][i] - earns.loc[rowIndex, 'buy'] 
            earns.loc[rowIndex, 'sell time'] = data['open time'][i]
            continue
        else:
            sigPriceBuy.append(np.nan)
            sigPriceSell.append(np.nan)
            continue

#     #SELL
#     if flag == 'bought':
#         last_40_prices = data.iloc[i - 41: i - 1]['close']
#         higher_price = max(last_40_prices)
        
#         rowIndex = earns[earns['buy'].notna() & earns['sell'].isna()].index.item()
        
#         if data['close'][i] > higher_price:
            
#             sigPriceBuy.append(np.nan)
#             sigPriceSell.append(data['close'][i])
#             flag = 'sold'
#             earns.loc[rowIndex, 'sell'] = data['close'][i]
#             earns.loc[rowIndex, 'earn'] = data['close'][i] - earns.loc[rowIndex, 'buy'] 
#             earns.loc[rowIndex, 'sell time'] = data['open time'][i]
#             continue
            
#         variation = (data['close'][i] / earns['buy'][rowIndex]) - 1
            
#         if variation <= -0.01:
#             sigPriceBuy.append(np.nan)
#             sigPriceSell.append(data['close'][i])
#             flag = 'sold'
#             earns.loc[rowIndex, 'sell'] = data['close'][i]
#             earns.loc[rowIndex, 'earn'] = data['close'][i] - earns.loc[rowIndex, 'buy'] 
#             earns.loc[rowIndex, 'sell time'] = data['open time'][i]
#             continue
        
#         sigPriceBuy.append(np.nan)
#         sigPriceSell.append(np.nan)
#         continue






    is_higher_than_sma150 = current_value['close'] > current_value['sma150']
    is_sar_lower = current_value['close'] > current_value['sar'] 
    is_positive_macdhist = current_value['macdhist'] > 0.1
    is_macd_up = current_value['macd'] > current_value['macdsignal']
    is_positive_roc = current_value['roc'] > 0

    preenchidos = earns[earns['buy'].notna() & earns['sell'].notna()]
    
    
    if len(preenchidos) > 0:
        # last_row = preenchidos.iloc[-1]['sell time']
        # minutes_diff = (current_value['open time'] - last_row).total_seconds() / 60
        
        last_row = preenchidos.iloc[-1]['sell time']
        sell_time = datetime.strptime(last_row, '%Y-%m-%d %H:%M:%S')
        current_rime = datetime.strptime(current_value['open time'], '%Y-%m-%d %H:%M:%S')
        minutes_diff = (current_rime - sell_time).total_seconds() / 60
        
        if minutes_diff < 20:
            sigPriceBuy.append(np.nan)
            sigPriceSell.append(np.nan)
            continue

        
    
    if is_higher_than_sma150 and is_sar_lower and (is_positive_macdhist and is_macd_up) and is_positive_roc: 
        if flag == 'sold':
            sigPriceBuy.append(data['close'][i])
            sigPriceSell.append(np.nan)
            flag = 'bought'
            earns = earns.append({ 
                'buy': data['close'][i],
                'buy time': data['open time'][i]
            }, ignore_index=True)
            continue
        else:
            sigPriceBuy.append(np.nan)
            sigPriceSell.append(np.nan)
            continue
    else:
        sigPriceBuy.append(np.nan)
        sigPriceSell.append(np.nan)
                
        
earns.to_csv("earns.csv", index=False)        
print('Lucro', earns['earn'].sum())
total = earns['earn'].count()
print('Quantidade', earns['earn'].count())
wins = earns[earns['earn'] > 0].count()
p = (wins/total)['earn'] * 100
print(f'Porcentagem {p}%')
print('-------')

Lucro 144.90259999999938
Quantidade 514
Porcentagem 46.69260700389105%
-------


In [ ]:
# df = dataframe.iloc[0:10]
# print(df)
# plt.figure(figsize = (50, 12.5))
# plt.plot(df['open time'], df['close'], label = 'BNB/BTC', alpha=0.30)
# plt.plot(df['open time'], df['sma3'], label = 'SMA 30', alpha=0.30)
# plt.plot(df['open time'], df['sma9'], label = 'SMA 30', alpha=0.30)
# plt.plot(df['open time'], df['sma20'], label = 'SMA 30', alpha=0.30)
# plt.plot(df['open time'], df['sma150'], label = 'SMA 100', alpha=0.30)
# plt.scatter(df['open time'], df['buy'], label = 'Buy', marker = '^', color = 'green')
# plt.scatter(df['open time'], df['sell'], label = 'Sell', marker = 'v', color = 'red')
# plt.title('BNB/BTC Close price')
# plt.xlabel('Date')
# plt.ylabel('Close price')
# # plt.legend(loc='upper left')
# plt.show()

In [ ]:
earns = earns[earns['buy'].notna() | earns['sell'].notna()]
earns.to_csv("result.csv", index=False)